# Created in this Notebook:

** P(NonDefault | race) **
- ProbNonDefaultGivenRace

** P(NonDefault | Score=x, race) **
- ProbNonDefaultGivenScoreEqualsXAndRace

** P(Default | Score=x, race) **
- ProbDefOrNotGivenScoreEqualsXAndRace

** P(race) **
- ProbOfBeingRace

** P(Score=x | race) **
- ProbScoreEqualsXGivenRace

** P(score=x & good & race) **
- ProbScoreEqualsXAndGoodAndRace

** P(NonDefault & race) **
- ProbNonDefaultAndRace

** P(Score=x | NonDefault, race) **
- ProbScoreEqualsXGivenNonDefaultAndRace

** P(Score>=x | NonDefault, race) **
- ProbScoreGreaterThanXGivenNonDefaultAndRace
- to obtain this, I calculated:
    - <b>P(score=x & NonDefault & race)</b>
    - <b>P(race & score=x) </b>
    - <b>P(NonDefault and race)</b>

** P(NonDefault | Score=>x, race) **
- ProbNonDefaultGivenScoreGreaterXAndRace
- This is supposed to yield the same results that ProbLoanReceiverIsGood.csv did
- Small rounding errors are causing this dataset to create different final results - unsure the cause

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
CumulativePercentageByDefaulters = pd.read_csv("Figure7A-fixed.csv")
CumulativePercentageByDemographic = pd.read_csv("Figure3A.csv")

<hr/>

# Calculating P(NonDefault | race) - pi values
- Named ProbNonDefaultGivenRace.csv

In [3]:
def getPisNonCumulative(dataset, raceSet, goodName, badName):
    good = dataset.set_index("Score")[goodName].diff().fillna(value=0)
    bad = dataset.set_index("Score")[badName].diff().fillna(value=0)
    total = raceSet.set_index("Score")["Percentage"].diff().fillna(value=0)
    return ((total - bad) / (good - bad)).fillna(value=0)

In [4]:
# Get the dataframe that holds the cumulative percentage, by demographic group
# This function is just for reorganizing the given data
def getPD(data, col, raceName):
    pd = data["Score"].to_frame(name="Score")
    race = np.full(len(data), raceName)
    pd["Demographic"] = race
    pd["Percentage"] = data[col]
    return pd

whites = getPD(CumulativePercentageByDemographic, "White", "white")
blacks = getPD(CumulativePercentageByDemographic, "Black", "black")
asians = getPD(CumulativePercentageByDemographic, "Asian", "asian")
hispanics = getPD(CumulativePercentageByDemographic, "Hispanic", "hispanic")

In [5]:
whitePi = getPisNonCumulative(CumulativePercentageByDefaulters, whites, "White (NonDefault)", "White (Default)")
blackPi = getPisNonCumulative(CumulativePercentageByDefaulters, blacks, "Black (NonDefault)", "Black (Default)")
asianPi = getPisNonCumulative(CumulativePercentageByDefaulters, asians, "Asian (NonDefault)", "Asian (Default)")
hispanicPi = getPisNonCumulative(CumulativePercentageByDefaulters, hispanics, "Hispanic (NonDefault)", "Hispanic (Default)")

In [6]:
whitePi[50], blackPi[50], asianPi[50], hispanicPi[50]

(0.77551020408163152,
 0.29896907216495305,
 0.80645161290323375,
 0.54545454545453709)

In [7]:
# hardcoded pi values to account for rounding error
pis = [0.759185,0.315164,0.550595,0.80066]
# pis = [whitePi[50], blackPi[50], asianPi[50], hispanicPi[50]]
ProbNonDefaultGivenRace = pd.DataFrame(data=[pis], columns=['white', 'black', 'hispanic', 'asian'])
ProbNonDefaultGivenRace.set_index("white").to_csv("ProbNonDefaultGivenRace.csv")

<hr/>

# Calculating P(NonDefault | Score=x, race) and P(Default | Score=x, race)

<h2 align='center'>$\frac{pi * P(Score=x | NonDefault)}{pi * P(Score=x|NonDefault) + (1-pi) * P(Score=x | Default)}$</h2>
- from our discussion / email about Bayes' Rule on April 10th
- CumulativePercentageByDefaulters (from Figure 7A) gives us P(Score>=x | NonDefault) and P(Score>=x | Default).
- https://www.federalreserve.gov/boarddocs/rptcongress/creditscore/figtables7.htm#d7A
- Using diff(), we can obtain P(Score=x|NonDefault) and P(Score=x|Default).
- Together, with the pi values calculated above. We have the necessary requirements to calculate P(NonDefault | Score=x)
- used to be named NonCumulativeProbabilities.csv (the one we weren't sure the origination of last week's meeting)

In [8]:
def getProbDefOrNotGivenScoreEqualsX(dataset, names, pis):
    probabilities = pd.DataFrame(index=dataset.index)
    for i in range(len(names)):
        nonDefault = dataset[names[i] + ' (NonDefault)'].diff().fillna(value=0)
        default = dataset[names[i] + ' (Default)'].diff().fillna(value=0)
        finalProbability = (
            (pis[i] * nonDefault) / ((pis[i] * nonDefault) + ((1 - pis[i])*(default))))
        probabilities['P(NonDefault|Score=x, ' + names[i] + ')'] = finalProbability.fillna(value=0)
        probabilities['P(Default|Score=x, ' + names[i] + ')'] = (1 - finalProbability.fillna(value=0))
    probabilities['Score'] = dataset.index
    return probabilities.set_index('Score')

In [9]:
ProbDefOrNotGivenScoreEqualsXAndRace = getProbDefOrNotGivenScoreEqualsX(
    CumulativePercentageByDefaulters.set_index('Score'),
    ["White", "Black", "Hispanic", "Asian"],
    pis)

In [10]:
ProbDefOrNotGivenScoreEqualsXAndRace.head()

,"P(NonDefault|Score=x, White)","P(Default|Score=x, White)","P(NonDefault|Score=x, Black)","P(Default|Score=x, Black)","P(NonDefault|Score=x, Hispanic)","P(Default|Score=x, Hispanic)","P(NonDefault|Score=x, Asian)","P(Default|Score=x, Asian)"
Score,,,,,,,,
0.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
0.5,0.000000,1.000000,0.004872,0.995128,0.021042,0.978958,0.000000,1.000000
1.0,0.023442,0.976558,0.023545,0.976455,0.011400,0.988600,0.058622,0.941378
1.5,0.024990,0.975010,0.019935,0.980065,0.000000,1.000000,0.000000,1.000000
2.0,0.038610,0.961390,0.022100,0.977900,0.025943,0.974057,0.101215,0.898785


<hr/>

# Creating P(race)
- taken from Table 9. 
- https://www.federalreserve.gov/boarddocs/rptcongress/creditscore/datamodel_tables.htm

In [11]:
sizes = [133165, 18274, 14702, 7906]
total = sum(sizes)
ProbOfBeingRace = pd.DataFrame(
    { 
    'Demographic' : ['white', 'black', 'hispanic', 'asian'],
    'P(race)' : [sizes[0]/total, sizes[1]/total, sizes[2]/total, sizes[3]/total]
    },
    columns=["Demographic", "P(race)"]
)

ProbOfBeingRace.set_index('Demographic').to_csv('ProbOfBeingRace.csv')

<hr/>

# Creating P(Score=x | race)
- taken from Table 3A. 
- https://www.federalreserve.gov/boarddocs/rptcongress/creditscore/figtables3.htm#d3A
- using the diff values

In [12]:
# used to be ProbOfBeingScore
# I'll give them this in a csv
ProbScoreEqualsXGivenRace = (
    pd.read_csv("figure3A.csv")
    .set_index(["Score"]).diff().fillna(0) / 100
)

ProbScoreEqualsXGivenRace.to_csv('ProbScoreEqualsXGivenRace.csv')

In [13]:
ProbScoreEqualsXGivenRace.head()

,White,Black,Hispanic,Asian
Score,,,,
0.0,0.0000,0.0000,0.0000,0.0000
0.5,0.0025,0.0112,0.0046,0.0013
1.0,0.0090,0.0414,0.0175,0.0075
1.5,0.0027,0.0114,0.0052,0.0019
2.0,0.0037,0.0142,0.0075,0.0025


# 'ProbScoreEqualsXGivenRace Differ' - this is where I see their difference before exporting to a csv, they are definitely differing by more than 0.0

In [14]:
ProbFromOther = pd.read_csv('ProbScoreEqualsXGivenRace-old.csv')

for race in ["White", "Black", "Hispanic", "Asian"]:
    print(race, (ProbFromOther[race.lower()] - ProbScoreEqualsXGivenRace[race]).abs().sum())

White 0.22900000000000045
Black 0.4477999999999998
Hispanic 0.3226999999999991
Asian 0.28189999999999954


<hr/>

# Calculating P(score=x | NonDefault, race)

### P(score=x | NonDefault, race) = P(score=x & NonDefault & race) / P(NonDefault and race)
<hr/>
### Step 1:
### P(score=x & NonDefault & race ) = P(race & score=x) * P(NonDefault | race, score=x)
###  P(score=x & NonDefault & race ) = P(race) * P(score=x | race) * P(NonDefault | race, score=x)
<hr/>
### Step 2:
### P(NonDefault and race) = P(race) * P(NonDefault | race)
<hr/>
### Step 3:
### Step 1 / Step2
### P(score=x | NonDefault, race) = P(score=x & NonDefault & race) / P(NonDefault and race)
<hr/>

## Step 1: Calculate P(Score=x & NonDefault & race)

In [15]:
ProbOfBeingRace.set_index('Demographic', inplace=True)

In [16]:
ProbScoreEqualsXGivenRace.head()

,White,Black,Hispanic,Asian
Score,,,,
0.0,0.0000,0.0000,0.0000,0.0000
0.5,0.0025,0.0112,0.0046,0.0013
1.0,0.0090,0.0414,0.0175,0.0075
1.5,0.0027,0.0114,0.0052,0.0019
2.0,0.0037,0.0142,0.0075,0.0025


In [17]:
ProbRaceAndScoreEqualsX = pd.DataFrame({
    'white': ProbOfBeingRace.loc['white']['P(race)'] * ProbScoreEqualsXGivenRace['White'],
    'asian': ProbOfBeingRace.loc['asian']['P(race)'] * ProbScoreEqualsXGivenRace['Asian'],
    'black': ProbOfBeingRace.loc['black']['P(race)'] * ProbScoreEqualsXGivenRace['Black'],
    'hispanic': ProbOfBeingRace.loc['hispanic']['P(race)'] * ProbScoreEqualsXGivenRace['Hispanic'],
})

In [18]:
ProbFromOther = pd.read_csv('ProbRaceAndScoreEqualsX.csv')
for race in ["white", "black", "hispanic", "asian"]:
    print(race, (ProbFromOther[race.lower()] - ProbRaceAndScoreEqualsX[race]).abs().sum())

white 0.17521005820266966
black 0.04701659436818787
hispanic 0.027258932357351672
asian 0.012805169867909215


In [19]:
ProbNonDefaultGivenScoreEqualsXAndRace = (
    ProbDefOrNotGivenScoreEqualsXAndRace[['P(NonDefault|Score=x, White)', 
                                 'P(NonDefault|Score=x, Black)',
                                'P(NonDefault|Score=x, Hispanic)',
                                'P(NonDefault|Score=x, Asian)']])

In [20]:
ProbNonDefaultGivenScoreEqualsXAndRace.head()

,"P(NonDefault|Score=x, White)","P(NonDefault|Score=x, Black)","P(NonDefault|Score=x, Hispanic)","P(NonDefault|Score=x, Asian)"
Score,,,,
0.0,0.000000,0.000000,0.000000,0.000000
0.5,0.000000,0.004872,0.021042,0.000000
1.0,0.023442,0.023545,0.011400,0.058622
1.5,0.024990,0.019935,0.000000,0.000000
2.0,0.038610,0.022100,0.025943,0.101215


In [21]:
ProbScoreEqualsXAndGoodAndRace = pd.DataFrame(index=ProbRaceAndScoreEqualsX.index)
ProbScoreEqualsXAndGoodAndRace['P(Score=x and NonDefault and White)'] = (
    ProbRaceAndScoreEqualsX['white'] * ProbNonDefaultGivenScoreEqualsXAndRace['P(NonDefault|Score=x, White)'])
ProbScoreEqualsXAndGoodAndRace['P(Score=x and NonDefault and Black)'] = (
    ProbRaceAndScoreEqualsX['black'] * ProbNonDefaultGivenScoreEqualsXAndRace['P(NonDefault|Score=x, Black)'])
ProbScoreEqualsXAndGoodAndRace['P(Score=x and NonDefault and Hispanic)'] = (
    ProbRaceAndScoreEqualsX['hispanic'] * ProbNonDefaultGivenScoreEqualsXAndRace['P(NonDefault|Score=x, Hispanic)'])
ProbScoreEqualsXAndGoodAndRace['P(Score=x and NonDefault and Asian)'] = (
    ProbRaceAndScoreEqualsX['asian'] * ProbNonDefaultGivenScoreEqualsXAndRace['P(NonDefault|Score=x, Asian)'])
# i'll give them this in a csv I think
ProbScoreEqualsXAndGoodAndRace.head()

,P(Score=x and NonDefault and White),P(Score=x and NonDefault and Black),P(Score=x and NonDefault and Hispanic),P(Score=x and NonDefault and Asian)
Score,,,,
0.0,0.000000,0.000000,0.000000,0.000000
0.5,0.000000,0.000006,0.000008,0.000000
1.0,0.000161,0.000102,0.000017,0.000020
1.5,0.000052,0.000024,0.000000,0.000000
2.0,0.000109,0.000033,0.000016,0.000011


In [22]:
# delete this later it will mess stuff up
ProbScoreEqualsXAndGoodAndRace.columns=['white', 'black', 'hispanic', 'asian']

In [23]:
ProbFromOther = pd.read_csv('P(score=xandgoodandrace).csv')

for race in ["white", "black", "hispanic", "asian"]:
    print(race, (ProbFromOther[race.lower()] - ProbScoreEqualsXAndGoodAndRace[race]).abs().sum())

white 0.18305259216161024
black 0.014612668194484476
hispanic 0.017668985457229308
asian 0.012851035980292675


In [24]:
ProbScoreEqualsXAndGoodAndRace.columns=['P(Score=x and NonDefault and White)', 'P(Score=x and NonDefault and Black)','P(Score=x and NonDefault and Hispanic)' ,'P(Score=x and NonDefault and Asian)']

## Step 2: Calculate P(NonDefault and race)

In [25]:
ProbGoodAndRace = pd.DataFrame({
    'white': ProbOfBeingRace.loc['white']['P(race)'] * ProbNonDefaultGivenRace['white'],
    'asian': ProbOfBeingRace.loc['asian']['P(race)'] * ProbNonDefaultGivenRace['asian'],
    'black': ProbOfBeingRace.loc['black']['P(race)'] * ProbNonDefaultGivenRace['black'],
    'hispanic': ProbOfBeingRace.loc['hispanic']['P(race)'] * ProbNonDefaultGivenRace['hispanic'],
})

In [26]:
# I'll give them this csv I think
ProbGoodAndRace

,asian,black,hispanic,white
0,0.03637,0.033091,0.04651,0.58086


## Step 3: Calculate P(Score=x | NonDefault, race)

In [27]:
ProbScoreEqualsXGivenNonDefaultAndRace = pd.DataFrame({
    'white': ProbScoreEqualsXAndGoodAndRace['P(Score=x and NonDefault and White)'] / ProbGoodAndRace['white'].values[0],
    'asian': ProbScoreEqualsXAndGoodAndRace['P(Score=x and NonDefault and Asian)'] / ProbGoodAndRace['asian'].values[0],
    'black': ProbScoreEqualsXAndGoodAndRace['P(Score=x and NonDefault and Black)'] / ProbGoodAndRace['black'].values[0],
    'hispanic': ProbScoreEqualsXAndGoodAndRace ['P(Score=x and NonDefault and Hispanic)'] / ProbGoodAndRace['hispanic'].values[0],
})


ProbScoreEqualsXGivenNonDefaultAndRace.head()

,asian,black,hispanic,white
Score,,,,
0.0,0.000000,0.000000,0.000000,0.000000
0.5,0.000000,0.000173,0.000176,0.000000
1.0,0.000549,0.003093,0.000362,0.000278
1.5,0.000000,0.000721,0.000000,0.000089
2.0,0.000316,0.000996,0.000353,0.000188


In [28]:
ProbFromOther = pd.read_csv('ProbScoreEqualsXGivenGoodAndRace.csv')

for race in ["white", "black", "hispanic", "asian"]:
    print(race, (ProbFromOther[race.lower()] - ProbScoreEqualsXGivenNonDefaultAndRace[race]).abs().sum())

white 0.3151408578970133
black 0.44159672153396745
hispanic 0.3799001574388349
asian 0.3533456418916701


<hr/>

# Calculating P(Score>=x | NonDefault, race)

In [29]:
ProbScoreGreaterThanXGivenNonDefaultAndRace = ProbScoreEqualsXGivenNonDefaultAndRace.iloc[::-1].cumsum()[::-1]
ProbScoreGreaterThanXGivenNonDefaultAndRace.head()

,asian,black,hispanic,white
Score,,,,
0.0,0.996122,1.026501,1.004775,0.989761
0.5,0.996122,1.026501,1.004775,0.989761
1.0,0.996122,1.026328,1.004600,0.989761
1.5,0.995573,1.023235,1.004237,0.989483
2.0,0.995573,1.022514,1.004237,0.989394


<hr/>

# Calculating P(NonDefault | Score=>x, race)
# THIS IS THE ONE THAT SOMETHING IS WRONG WITH UNFORTUNATELY
### Should match ProbLoanReceiverIsGood.csv, but does not match

<h2 align='center'>$\frac{P(NonDefault and Score>=x | race)}{P(Score>=x | race)}$</h2>

** P(NonDefault | Score=x, race) **
- calculated earlier in this notebook
- ProbNonDefaultGivenScoreEqualsXAndRace

** P(NonDefault and Score=x | race) = P(NonDefault | Score=x, race) * P(Score=x | race) **
- ProbNonDefaultGivenScoreEqualsXAndRace * ProbScoreEqualsXAndRace (calculated above)
- we can use cumsum() on the reverse of this to obtain P(NonDefault and Score>=x | race)

** P(NonDefault and Score>=x | race) **
- ProbNonDefaultAndScoreGreaterXGivenRace
- see above for how to calculate this

** P(Score>=x | race) **
- We already calculated P(Score=x | race), we can use cumsum() on the reverse of ProbScoreEqualsXGivenRace to obtain this

In [30]:
ProbNonDefaultGivenScoreEqualsXAndRace.columns= ['Asian', 'Black', 'Hispanic', 'White']
ProbNonDefaultAndScoreEqualsXGivenRace = (
    ProbScoreEqualsXGivenRace * ProbNonDefaultGivenScoreEqualsXAndRace)
ProbNonDefaultAndScoreGreaterXGivenRace = (
    ProbNonDefaultAndScoreEqualsXGivenRace.iloc[::-1].cumsum()[::-1])

In [31]:
ProbScoreGreaterXGivenRace = ProbScoreEqualsXGivenRace.iloc[::-1].cumsum()[::-1]

In [32]:
ProbNonDefaultGivenScoreGreaterXAndRace = (
    ProbNonDefaultAndScoreGreaterXGivenRace / ProbScoreGreaterXGivenRace)

In [33]:
ProbNonDefaultGivenScoreGreaterXAndRace.columns = ProbNonDefaultGivenScoreGreaterXAndRace.columns.str.lower()

In [34]:
ProbNonDefaultGivenScoreGreaterXAndRace.head()

,asian,black,hispanic,white
Score,,,,
0.0,0.787367,0.323743,0.553280,0.764245
0.5,0.787367,0.323743,0.553280,0.764245
1.0,0.788392,0.327357,0.555739,0.766160
1.5,0.794180,0.340643,0.565482,0.772603
2.0,0.795657,0.344552,0.568505,0.774719


In [35]:
# This is the dataset that I was attempting to recreate, though I believe
# that rounding is causing the discrepencies
pd.read_csv('ProbLoanReceiverIsGood.csv').head()

,TransRisk Score,asian,black,hispanic,white
0,0.0,0.800660,0.315384,0.550650,0.759261
1,0.5,0.800660,0.315384,0.550650,0.759261
2,1.0,0.801702,0.318908,0.553099,0.761164
3,1.5,0.807309,0.331898,0.562799,0.767862
4,2.0,0.808860,0.335718,0.565808,0.769890


In [36]:
# THEY NEED THIS CSV IN THE TUTORIAL
ProbNonDefaultGivenScoreGreaterXAndRace.to_csv('ProbNonDefaultGivenScoreGreaterXAndRace.csv')